In [2]:
import arviz as az
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pystan
import seaborn as sns
import scipy as sp

#### Import stan convergence utilities from local directory 
Utilities originally by Betancourt, [see this notebook on Bayesian workflow](http://mc-stan.org/users/documentation/case-studies/pystan_workflow.html).

In [3]:
%run utility/stan_utility.py

#### Methods for saving and loading models

In [5]:
def save_model(model, filename):
    """Saves the compiled model to file."""
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

def load_model(filename):
    """Load already compiled model from file."""
    return pickle.load(open(filename, 'rb'))

## __3. Model Comparison__
Load all models.


In [18]:
model_filenames = ['lin_3_uninformative.stan.saved',
                   'lin_3_informative.stan.saved',
                   'lin_5_informative.stan.saved',
                   'lin_5_informative-interaction.stan.saved']

model_names = ['3 Predictors Uninformative',
               '3 Predictors Informative, Normal',
               '5 Predictors Informative, Normal',
               '5 Predictors Informative inter, Normal']

models = [load_model(M_name) for M_name in model_filenames]

ModuleNotFoundError: No module named 'stanfit4anon_model_292039ab535cf44cf201c28c58f22d90_4516776706501392494'

Calculate PSIS-LOO-values:

In [17]:
params = ['α','β_1','β_2','β_3','β_4','β_5','β_6','β_7','sigma']
params_mean =  [e + ',μ' for e in params]
params_stds =  [e + ',σ' for e in params]
df = pd.DataFrame(columns=[params_mean + params_stds])

In [251]:
from functools import reduce
loos = []
params = ['α','β_1','β_2','β_3','β_4','β_5','β_6','β_7','sigma']

for M in models:
    azfit = az.from_pystan(fit=M, prior=prior_dict, 
                           observed_data='y', 
                           posterior_predictive='ypred', 
                           log_likelihood='log_lik')
    df_M = az.loo(azfit)
    
    m_s = []
    for func in [lambda X: np.mean(X), lambda X: np.var(X)]:
        m_s.append([func(M['a'])] + [func(M['b'][:,i]) for i in range(M['b'].shape[1])] + [func(M['sigma'])])
    
    df_M = pd.concat([df_M,df],axis=1)
    df_M.loc[params_mean] = m_s[0]
    df_M.loc[params_stds] = m_s[1]
    loos.append(df_M)

loos_merged = reduce(lambda left,right: pd.concat([left,right], axis=0), loos)

/home/mrreflex/anaconda3/envs/bdaproject/lib/python3.7/site-packages/arviz/stats/stats.py:372: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for
        one or more samples. You should consider using a more robust model, this is because
        importance sampling is less likely to work well if the marginal posterior and LOO posterior
        are very different. This is more likely to happen with a non-robust model and highly
        influential observations.
  influential observations."""


KeyError: "['α | μ' 'β_1 | μ' 'β_2 | μ' 'β_3 | μ' 'β_4 | μ' 'β_5 | μ' 'β_6 | μ'\n 'β_7 | μ' 'sigma | μ'] not in index"

In [250]:
loos_merged.set_axis(model_names, axis=0, inplace=True)
loos_merged

,loo,loo_se,p_loo,warning,"(α | μ,)","(β_1 | μ,)","(β_2 | μ,)","(β_3 | μ,)","(β_4 | μ,)","(β_5 | μ,)",...,"(sigma | μ,)","(α | σ,)","(β_1 | σ,)","(β_2 | σ,)","(β_3 | σ,)","(β_4 | σ,)","(β_5 | σ,)","(β_6 | σ,)","(β_7 | σ,)","(sigma | σ,)"
3 Predictors Uninformative,-5096.359694,5532.216878,7362.516070,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"3 Predictors Informative, Normal",-6133.387852,5467.369648,6585.259986,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"5 Predictors Informative, Normal",-4655.450268,5840.526255,7195.049404,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"5 Predictors Informative inter, Normal",-3597.043730,5961.645286,7682.618950,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Use leave-one-out cross validation (LOO-CV) to assess the predictive performance of the different models.

>* PSIS-LOO values, the effective number of parameters peff , and the k-values for each of the
three models

>* an assessment of how reliable the PSIS-LOO estimates are for the three models based on
the k-values

>* an assessment of whether there are differences between the models, and if so, which model
should be selected according to PSIS-LOO

>* number of effective parameters

## __4. Conclusions__
>  Even the coin tosses and die rolls ubiquitous in probability theory texts are not truly exchangeable. The more relevant question is, ‘Do the model’s deficiencies have a noticeable effect on the substantive inferences?’
p 142


>More formally, we can check a model by external validation using the model to make predic-
tions about future data, and then collecting those data and comparing to their predictions.
Posterior means should be correct on average, 50% intervals should contain the true values
half the time, and so forth. p. 143

See the hierarchical regression model: 142->

__Notes:__
Would the results be easier to interpret if we would normalize the streams to interval [0,1]?

We need to conduct proper sensitivity analysis using different priors and try to develop intuition about the data / hypothesize with more clarity. How to formulate reasonable priors for quite arbitrary linear coefficients $\beta$? Explaining the steepness of slope... 

Also we should think about how to construct hierarchical priors based on earlier data (weekly top 100). Also develop understanding about the make-up off global data - differing countries with differing cultures. For example chinese or italian music could be quite different compared to UK music. We should also train on Finnish and Swedish data and compare cultural differences by comparing posteriors. 